In [1]:
# Import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
import os
import tempfile
from qiskit import *
from qiskit.tools.visualization import plot_histogram
from qiskit.algorithms import Grover
from qiskit.primitives import Sampler
from qiskit.circuit.library import PhaseOracle
from qiskit.exceptions import MissingOptionalLibraryError
from qiskit.algorithms.amplitude_amplifiers import AmplificationProblem
import numpy as np
from qiskit.result import utils
from qiskit.utils import QuantumInstance
pi = np.pi # pi
TOKEN="0f466da63a29ab5932d1e030884099b76de6be3cc5639ee1a4a5f44922ac97b86a9a2a5b2f523d1c85023dffa09296293717e2f6a3f8b852f3cf337dab5eed07"
IBMQ.save_account(TOKEN)
IBMQ.load_account()

C:\Users\Keshav\anaconda3\envs\MachineLearning\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  app.launch_new_instance()
C:\Users\Keshav\anaconda3\envs\MachineLearning\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  app.launch_new_instance()
configrc.store_credentials:WARNING:2023-04-02

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [2]:
D = 17
M = 5
N = ((D*M) % 254) + 1
# Print the N in binary
print("N = ", bin(N)[2:].zfill(8))
N = (bin(N)[2:].zfill(8), 2)[0]
print(N)

N =  01010110
01010110


In [28]:
function_values = []
for i in range(0,8):
    function_values.append(N[7-i])

for i in range(0,8):
    print("i = ", bin(i)[2:].zfill(3), "f(i) = ", function_values[i])

print(function_values)
temp = ""
for i in function_values:
    temp+=i
print(temp)
function_values = temp

i =  000 f(i) =  0
i =  001 f(i) =  1
i =  010 f(i) =  1
i =  011 f(i) =  0
i =  100 f(i) =  1
i =  101 f(i) =  0
i =  110 f(i) =  1
i =  111 f(i) =  0
['0', '1', '1', '0', '1', '0', '1', '0']
01101010


In [12]:
# function to convert a bitstring into a boolean expression
def get_expression(bitstring):
    expression = ""
    count = 0
    for i in range(len(bitstring)):
        j = bin(i)[2:].zfill(3)
        if(bitstring[i] == '1'):
            count += 1
            if (count == 1):
                expression += "("
            else:
                expression += " | ("
            if (j[2] == "0"):
                expression += "~c"
            else:
                expression += "c"
            expression += " & "
            if (j[1] == "0"):
                expression += "~b"
            else:
                expression += "b"
            expression += " & "
            if (j[0] == "0"):
                expression += "~a"
            else:
                expression += "a"

            expression += ")"
        else:
            pass  # do nothing

    return expression


print(get_expression("11000000"))


(~c & ~b & ~a) | (c & ~b & ~a)


In [25]:
 # a = b1 ,b = b2, c = b3
def apply_grover(bitsring_rep,iter):
    expression = get_expression(bitsring_rep)
    try:
        oracle = PhaseOracle(expression)
        problem = AmplificationProblem(oracle)
        provider = IBMQ.get_provider(project="intro-quantum-co")
        backend = provider.get_backend("ibmq_qasm_simulator")
        quantum_instance = QuantumInstance(backend=backend,shots = 100)
        grover = Grover(quantum_instance=quantum_instance,iterations=iter)
        result = grover.amplify(problem)
        # Return the top result in decimal
        return int(result.top_measurement,2)
    except MissingOptionalLibraryError as ex:
        print(ex)
print(apply_grover("10000000",1))

7


In [23]:
def boyer_search(fsr):
    m = 1
    l = 4/3
    runs = 0
    steps = 0
    N = len(fsr)
    while (True):
        # j is uniformly random among the non-negative integers smaller than m
        j = np.random.randint(0, m+1)
        steps += 1
        # apply j iterations of grover's
        result = apply_grover(fsr, j)
        runs += j
        if (fsr[result]=='1'):
            return result,runs,steps
        else:
            m = min(l*m,np.sqrt(N))
        if steps > np.ceil(np.sqrt(N)):
            return -1,runs,steps


(0, 0, 1)

In [34]:
def custom_algorithm(original):
    working = original
    actual_solutions = 0
    solutions_found = 0
    # count the number of "1" in original string
    for i in range(len(original)):
        if (original[i] == '1'):
            actual_solutions += 1
    total_outputs = 0
    total_calls = 0
    precision = 0
    recall = 0
    while (True):
        try:
            _ = boyer_search(working)
        except:
            precision = solutions_found / total_outputs
            recall = solutions_found / actual_solutions
            print(working)
            return precision,recall,total_calls
        total_calls += _[1]
        total_outputs += _[2]
        if (_[0] == -1):
            precision = solutions_found / total_outputs
            recall = solutions_found / actual_solutions
            print(working)
            return precision,recall,total_calls
        else:
            solutions_found +=1
            idx = _[0]
            working = working[:idx] + '0' + working[idx+1:]

number_of_runs = 20
cache = []
for i in range(number_of_runs):
    _ = custom_algorithm(function_values)
    cache.append(_)
print("Number of runs",number_of_runs)
print("Avg Precision across runs",sum([_[0] for _ in cache])/number_of_runs)
print("Avg Recall across runs",sum([_[1] for _ in cache])/number_of_runs)
print("Avg Number of calls across runs",sum([_[2] for _ in cache])/number_of_runs)

00100010
00000010
01101010
00000000
00000000
00101000
01100000
00001000
01101000
00100000
01000000
01101010
00100010
01000010
00000000
00000000
00101000
00101000
01100010
00101010
Number of runs 20
Avg Precision across runs 0.31289682539682545
Avg Recall across runs 0.5625
Avg Number of calls across runs 3.65
